In [1]:
import os
print(f"Starting notebook in directory: {os.getcwd()}")
os.chdir("../")
print(f"Moving notebook to following directory: {os.getcwd()}")

Starting notebook in directory: c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project\research
Moving notebook to following directory: c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project


In [2]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [13]:
from src.cnnClassifier.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [4]:
import tensorflow as tf

In [20]:
class Evaluation:

    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
    
    
    def save_scores(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [21]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_scores()


except Exception as e:
    raise e

[2025-05-14 22:25:54,516: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-14 22:25:54,526: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-14 22:25:54,526: INFO: common: created directory at: artifacts]


Found 78 images belonging to 2 classes.


c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project\.chicken_venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - acc: 0.8721 - loss: 0.4243
[2025-05-14 22:26:05,237: INFO: common: json file saved at: scores.json]
